In [2]:
import jieba
import os
import pandas as pd
import numpy as np
import csv
import re
import chardet
import emoji
import sys
import unicodedata
import joblib
import seaborn as sns

In [3]:
from pandarallel import pandarallel

#并行初始化
pandarallel.initialize()

INFO: Pandarallel will run on 8 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.

https://nalepae.github.io/pandarallel/troubleshooting/


In [4]:
#加载数据
df = pd.read_csv(r'D:/0609-3/大数据研究/raw123.csv',usecols=['content','user_name','birthday','gender','time','school','created_at'],encoding='utf-8')

In [5]:
practise = df
practise.head()

,time,user_name,content,school,gender,birthday,created_at
0,2022/3/23 1:56,青春有你2-tora酱,！！！//@白日夣患者:<U+0001F64F>#上海疫情# <U+0001F64F>#闵行...,NaN,"\f\""""","\1923-07-10 巨蟹座\""""","\2014-03-14 18:53:37\"""""
1,2022/7/11 0:43,长肉有点难nn,！！晚上十点多停电了，国家电网和上海电力的电话一直打不进去，女房东不懂电路，男房东联系不上，...,NaN,"\f\""""","\1995-06-15 双子座\""""","\2017-01-11 22:35:41\"""""
2,2022/7/14 11:35,麦和的人字拖,# 电影人生大事##上海疫情# 我就想看个电影，距离劝退，周边的影院都还歇业呢 <U+200B>,NaN,"\f\""""","\1900-06-12 双子座\""""","\2014-03-13 21:14:40\"""""
3,2022/3/26 11:40,Ponytailsweet,#2岁# 2岁14天#上海疫情# 居家隔离 每天都要好好吃饭 2上海 <U+200B>,NaN,"\f\""""","\1992-08-14 狮子座\""""","\2010-03-01 20:07:44\"""""
4,2022/3/24 10:34,白桃甜酒么么君,#moの摸鱼日常##隔离日记##上海疫情# 昨天被通知要去做核酸准备今天做治疗。到了晚上八点...,NaN,"\f\""""","\1900-10-23 天秤座\""""","\2010-04-13 23:29:28\"""""


In [6]:
practise.dtypes

time          object
user_name     object
content       object
school        object
gender        object
birthday      object
created_at    object
dtype: object

 #数据预处理

In [17]:
import sys
sys.executable

'E:\\project\\Scripts\\python.exe'

In [18]:
#pip install ipykernel 更换内核

In [19]:
#ipython kernelspec list cmd打开查看列表

In [20]:
#pip install pyhanlp

In [21]:
#分词和数据清理
def load_stopwords(filepath):
    m_stopwords = [line.strip()for line in open(filepath,'r',encoding='utf-8').readlines()]
    return m_stopwords

#载入停用词表

stop_word_file = r'D:\0609-3\NLP\project\stop_words1893.txt'
stopwords = load_stopwords(stop_word_file)

In [22]:
stopwords[0]='!'
stopwords.append('##')
stopwords.append('链接')
stopwords.append('，')
stopwords.append('！')
stopwords.append('？')
new = (['网页','链接','上海','罗一舟','肖战','朱一龙','任嘉伦','邓伦','微博','视频','（','）','<','>','+','顾魏','特别','越来越','好像','我要','刚刚','天天','林之校','啊啊啊','王一博','翟潇闻','真的','哈哈哈','感觉',
       '不到','一点','呜呜','哈哈哈哈','好好','这是','时代','少年','余生','指教','本来','晚上','天天','不想','刘浩存','电视剧','）','（','.','。','.#','#','','：','@','岁','例','日','确诊','性别','收起全文','日至','月','新增','～','确诊','收起','全文','人類','飛行'])
stopwords.extend(new)

stopwords

['!',
 '"',
 '#',
 '$',
 '%',
 '&',
 "'",
 '(',
 ')',
 '*',
 '+',
 ',',
 '-',
 '--',
 '.',
 '..',
 '...',
 '......',
 '...................',
 './',
 '.一',
 '.数',
 '.日',
 '/',
 '//',
 '0',
 '1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 ':',
 '://',
 '::',
 ';',
 '<',
 '=',
 '>',
 '>>',
 '?',
 '@',
 'A',
 'Lex',
 '[',
 '\\',
 ']',
 '^',
 '_',
 '`',
 'exp',
 'sub',
 'sup',
 '|',
 '}',
 '~',
 '~~~~',
 '·',
 '×',
 '×××',
 'Δ',
 'Ψ',
 'γ',
 'μ',
 'φ',
 'φ．',
 'В',
 '—',
 '——',
 '———',
 '‘',
 '’',
 '’‘',
 '“',
 '”',
 '”，',
 '…',
 '……',
 '…………………………………………………③',
 '′∈',
 '′｜',
 '℃',
 'Ⅲ',
 '↑',
 '→',
 '∈［',
 '∪φ∈',
 '≈',
 '①',
 '②',
 '②ｃ',
 '③',
 '③］',
 '④',
 '⑤',
 '⑥',
 '⑦',
 '⑧',
 '⑨',
 '⑩',
 '──',
 '■',
 '▲',
 '',
 '、',
 '。',
 '〈',
 '〉',
 '《',
 '》',
 '》），',
 '」',
 '『',
 '』',
 '【',
 '】',
 '〔',
 '〕',
 '〕〔',
 '㈧',
 '一',
 '一.',
 '一一',
 '一下',
 '一个',
 '一些',
 '一何',
 '一切',
 '一则',
 '一则通过',
 '一天',
 '一定',
 '一方面',
 '一旦',
 '一时',
 '一来',
 '一样',
 '一次',
 '一片',
 '一番',
 '一直',
 '一致',
 '一般',
 '一起',
 '一转眼'

In [23]:
from pandarallel import pandarallel
df=df.copy()

NameError: name 'df' is not defined

In [24]:
#str 和unicode的互转
#coding=utf-8
def to_unicode(unicode_or_str):
    if isinstance(unicode_or_str,str):
        value=unicode_or_str.decode('utf-8')
    else:
        value=unicode_or_str
    return value
def to_str(unicode_or_str):
    value=unicode_or_str.encode('utf-8')
    return value

In [25]:
practise['content'] = to_unicode(practise.content)
stopwords = to_unicode(stopwords)

NameError: name 'practise' is not defined

In [26]:
def seg_sentence(sentence):
    sentence_seged = jieba.cut(sentence.strip())
    stop = stopwords # 这里加载停用词的路径
    outstr = ''
    for word in sentence_seged:
        if word not in stop:
            if word != '\t':
                outstr += word
                outstr += " "
    return outstr

In [27]:
'''
from harvesttext import HarvestText
ht = HarvestText()
for line1 in practise['content']:
    line0 = re.sub(r"#\S+#", "", line1) 
    cleaned_content = ht.clean_text(line0, emoji=False,remove_url=True,email=True,t2s=True)
    clean1 = cleaned_content.split('\n')
    out = []
    for line2 in clean1:
        line_seg = seg_sentence(line2) # 这里的返回值是字符串
        out.append(line_seg)
        #print(out)
        u_emoji = []
        for line3 in out:
            line_u = str(line3)
            line_un = emoji.replace_emoji(line_u,' ') # 这里的返回值是字符串
            line_u = re.sub('[a-zA-Z]',' ',line_un) #去掉英文字母
            line_f = re.sub(r'[0-9]+', ' ', line_u) #去掉数字
            #line_ff = re.sub('([^\u4e00-\u9fa5])', ' ', line_f) #去掉除中文外全部字符
            u_emoji.append(line_f)
    #with open(r'D:\0609-3\NLP\project\clean1.txt', 'w') as f:  # 设置文件对象
        #print(cleaned_content,file = f)
            name = ['clean']
            test=pd.DataFrame(columns=name,data=u_emoji) 
''''''

SyntaxError: incomplete input (3980682576.py, line 25)

In [28]:
from harvesttext import HarvestText
ht = HarvestText()

In [19]:
textclean=[]
for line0 in practise['content']:
    line1 = re.sub(r"#\S+#", "", line0) 
    line1 = ht.clean_text(line1, emoji=False,remove_url=True,email=True,t2s=True)
    textclean.append(line1)
#print("\n".join(textclean))

In [20]:
textclean

['！！！',
 '！！晚上十点多停电了，国家电网和上海电力的电话一直打不进去，女房东不懂电路，男房东联系不上，小区也是晚上突然封了，我好惨！！这么热的天停电可怎么过！',
 '# 电影人生大事 我就想看个电影，距离劝退，周边的影院都还歇业呢',
 '2岁14天 居家隔离 每天都要好好吃饭 2上海',
 '昨天被通知要去做核酸准备今天做治疗。到了晚上八点半车来了，下楼被两个大白要求穿隔离服，说我们是阳性病人。我们这栋楼有阳性，但是我们从来没接触过他们都很自觉的躲避大家。被大白一闹，司机也以为我们是阳性吓跑了。随后的两小时没有任何一辆车愿意来接，最后突然说你们走着去吧，那就走吧，和妈妈走了好远到了，护士非常惊讶我们自己来了，但是还是帮我们安排了检测。还召唤了一辆转运车一直到十二点我们才回到家。大白认真我可以理解，但是我是完全阴性的，非要阻止我正常就医，实在让人不可理喻。我们是封控楼的阴性住户，配合你们被关，还要被歧视吗？ 2上海 收起全文d',
 '平平安安，健健康康~',
 '一直奉行：吃我想吃的饭，见我想见的人看喜欢的风景，做喜欢的事去喜欢的地方看看不留遗憾 也不要遗憾想做什么就立马行动起来',
 '又想到了那个留学生网红，没有比较就没有伤害',
 '补了徐汇区 漏了闵行区补了闵行区 漏了嘉定区补了嘉定区 漏了浦东新区……是要轮流补一圈么',
 '她在英国做了三轮化疗，被大夫委婉拒绝了，才回国。下飞机就绿色通道进医院了啊专家组远程也开会了， 现在这个病人要求一整套专家组进驻到隔离区内实地会诊，但专家组手上还有很多国内癌症患者不可能为了她一个人进去出不来。于是现在开始舆论施压了？这就是相当于会哭的孩子有奶吃吗？ 首先把舆论闹大，让网民为她冲锋陷阵，再给政府施压，最后得到她想要的优待。没钱没特权的普通人求助无门，在雨中跪地祈求，更别提能见到专家，还只能闭环为她一个人治疗…这不就是网络版的“医闹”？？？ 祝她早日康复吧！ 2上海·陆家嘴 收起全文d',
 '蹲个0点宝山退出决赛圈 ！！！ 救命',
 '吊龙nice茶百道的生椰绝绝子周末愉快了 忘了那些不开心的事情…… 2上海',
 '闲着无聊的时候看看书、看看小视频、听听击球的声音…… Lrickyaoyao的微博视频',
 '上海在调适流动性与疫情防控中，反复琢磨着既要严控疫情，还要尽量最小的代价。显然这次病毒的传播

In [21]:
u_clean = []
for line in textclean:
    line_u1 = str(line)
    line_un1 = emoji.replace_emoji(line_u1,' ') # 这里的返回值是字符串
    line_u1 = re.sub('[a-zA-Z]',' ',line_un1) #去掉英文字母
    #line_f1 = re.sub(r"\s+", " ",line_u1 )
    line_f1 = re.sub(u"([^\u4e00-\u9fa5\u0030-\u0039\u0041-\u005a\u0061-\u007a])", ",", line_u1) #去除特殊符号
    line_ff1 = re.sub(r'收起全文$',",", line_f1)
    line_fff1 = re.sub(r'\w+[的微博视频]$',",", line_ff1)
    line_ffff1 = re.sub(r'2上海',",", line_fff1)
    line_f2 = re.sub(r'[,]+',",", line_ffff1)
    
    #line_f1 = re.sub(r'[0-9]+', ' ', line_u1) #去掉数字
    #line_ff1 = re.sub('([^\u4e00-\u9fa5])', ' ', line_f1) #去掉除中文外全部字符
    u_clean.append(line_f2)

In [22]:
u_clean

[',',
 ',晚上十点多停电了,国家电网和上海电力的电话一直打不进去,女房东不懂电路,男房东联系不上,小区也是晚上突然封了,我好惨,这么热的天停电可怎么过,',
 ',电影人生大事,我就想看个电影,距离劝退,周边的影院都还歇业呢',
 '2岁14天,居家隔离,每天都要好好吃饭,',
 '昨天被通知要去做核酸准备今天做治疗,到了晚上八点半车来了,下楼被两个大白要求穿隔离服,说我们是阳性病人,我们这栋楼有阳性,但是我们从来没接触过他们都很自觉的躲避大家,被大白一闹,司机也以为我们是阳性吓跑了,随后的两小时没有任何一辆车愿意来接,最后突然说你们走着去吧,那就走吧,和妈妈走了好远到了,护士非常惊讶我们自己来了,但是还是帮我们安排了检测,还召唤了一辆转运车一直到十二点我们才回到家,大白认真我可以理解,但是我是完全阴性的,非要阻止我正常就医,实在让人不可理喻,我们是封控楼的阴性住户,配合你们被关,还要被歧视吗,收起全文,',
 '平平安安,健健康康,',
 '一直奉行,吃我想吃的饭,见我想见的人看喜欢的风景,做喜欢的事去喜欢的地方看看不留遗憾,也不要遗憾想做什么就立马行动起来',
 '又想到了那个留学生网红,没有比较就没有伤害',
 '补了徐汇区,漏了闵行区补了闵行区,漏了嘉定区补了嘉定区,漏了浦东新区,是要轮流补一圈么',
 '她在英国做了三轮化疗,被大夫委婉拒绝了,才回国,下飞机就绿色通道进医院了啊专家组远程也开会了,现在这个病人要求一整套专家组进驻到隔离区内实地会诊,但专家组手上还有很多国内癌症患者不可能为了她一个人进去出不来,于是现在开始舆论施压了,这就是相当于会哭的孩子有奶吃吗,首先把舆论闹大,让网民为她冲锋陷阵,再给政府施压,最后得到她想要的优待,没钱没特权的普通人求助无门,在雨中跪地祈求,更别提能见到专家,还只能闭环为她一个人治疗,这不就是网络版的,医闹,祝她早日康复吧,陆家嘴,收起全文,',
 '蹲个0点宝山退出决赛圈,救命',
 '吊龙,茶百道的生椰绝绝子周末愉快了,忘了那些不开心的事情,',
 '闲着无聊的时候看看书,看看小视频,听听击球的声音,',
 '上海在调适流动性与疫情防控中,反复琢磨着既要严控疫情,还要尽量最小的代价,显然这次病毒的传播速度掩盖了治病率低的特点,传播快就带来恐慌,不仅是上海还有外省市,人们多考虑病毒有还是没有,不太注意病毒治愈不

In [23]:
name = ['content_clean']
test1=pd.DataFrame(columns=name,data=u_clean)

In [24]:
df1=pd.concat([practise,test1],axis=1)
df1.head()

,time,user_name,content,school,gender,birthday,created_at,content_clean
0,2022/3/23 1:56,青春有你2-tora酱,！！！//@白日夣患者:<U+0001F64F>#上海疫情# <U+0001F64F>#闵行...,NaN,"\f\""""","\1923-07-10 巨蟹座\""""","\2014-03-14 18:53:37\""""",","
1,2022/7/11 0:43,长肉有点难nn,！！晚上十点多停电了，国家电网和上海电力的电话一直打不进去，女房东不懂电路，男房东联系不上，...,NaN,"\f\""""","\1995-06-15 双子座\""""","\2017-01-11 22:35:41\""""",",晚上十点多停电了,国家电网和上海电力的电话一直打不进去,女房东不懂电路,男房东联系不上,小..."
2,2022/7/14 11:35,麦和的人字拖,# 电影人生大事##上海疫情# 我就想看个电影，距离劝退，周边的影院都还歇业呢 <U+200B>,NaN,"\f\""""","\1900-06-12 双子座\""""","\2014-03-13 21:14:40\""""",",电影人生大事,我就想看个电影,距离劝退,周边的影院都还歇业呢"
3,2022/3/26 11:40,Ponytailsweet,#2岁# 2岁14天#上海疫情# 居家隔离 每天都要好好吃饭 2上海 <U+200B>,NaN,"\f\""""","\1992-08-14 狮子座\""""","\2010-03-01 20:07:44\""""","2岁14天,居家隔离,每天都要好好吃饭,"
4,2022/3/24 10:34,白桃甜酒么么君,#moの摸鱼日常##隔离日记##上海疫情# 昨天被通知要去做核酸准备今天做治疗。到了晚上八点...,NaN,"\f\""""","\1900-10-23 天秤座\""""","\2010-04-13 23:29:28\""""","昨天被通知要去做核酸准备今天做治疗,到了晚上八点半车来了,下楼被两个大白要求穿隔离服,说我们..."


In [25]:
out = []
for line2 in u_clean:
        line_seg = seg_sentence(line2) # 这里的返回值是字符串
        out.append(line_seg)

Building prefix dict from the default dictionary ...
2023-04-28 09:40:23,663 : DEBUG : Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\ASUS\AppData\Local\Temp\jieba.cache
2023-04-28 09:40:23,663 : DEBUG : Loading model from cache C:\Users\ASUS\AppData\Local\Temp\jieba.cache
Loading model cost 0.863 seconds.
2023-04-28 09:40:24,526 : DEBUG : Loading model cost 0.863 seconds.
Prefix dict has been built successfully.
2023-04-28 09:40:24,526 : DEBUG : Prefix dict has been built successfully.


In [26]:
out

['',
 '十点 停电 国家电网 电力 电话 女房东 懂 电路 男 房东 不上 小区 封 好惨 热 天 停电 ',
 '电影 人生大事 想 看个 电影 距离 劝退 周边 影院 歇业 ',
 '14 天 居家 隔离 吃饭 ',
 '昨天 通知 做 核酸 做 治疗 八点半 车来 下楼 两个 大白 穿 隔离 服 说 阳性 病人 这栋 楼有 阳性 接触 自觉 躲避 大白 一闹 司机 阳性 吓跑 随后 两 小时 一辆车 来接 说 走 走 妈妈 走 远 护士 非常 惊讶 帮 安排 检测 召唤 一辆 转运 车 十二点 回到 家 大白 理解 阴性 非 阻止 就医 实在 不可理喻 封 控楼 阴性 住户 配合 被关 歧视 ',
 '平平安安 健健康康 ',
 '奉行 吃 想 吃 饭 想见 喜欢 风景 做 喜欢 事 喜欢 地方 不留 遗憾 遗憾 想 做 ',
 '想到 留学生 网红 伤害 ',
 '补 徐汇区 漏 闵行区 补 闵行区 漏 嘉定区 补 嘉定区 漏 浦东新区 轮流 补 一圈 ',
 '英国 做 三轮 化疗 大夫 委婉 拒绝 回国 飞机 绿色通道 进 医院 专家组 远程 开会 病人 一整套 专家组 进驻 隔离区 实地 会诊 专家组 手上 国内 癌症 患者 出不来 舆论 施压 相当于 哭 孩子 有奶 吃 首先 舆论 闹大 网民 冲锋陷阵 政府 施压 想要 优待 没钱 特权 普通人 求助无门 雨 中 跪 祈求 别提 见到 专家 只能 闭环 治疗 网络版 医闹 祝 早日康复 陆家嘴 ',
 '蹲个 点 宝山 退出 决赛圈 救命 ',
 '吊龙 茶百道 生椰绝 绝子 周末 愉快 忘 开心 事情 ',
 '闲着 无聊 看看书 小视频 听听 击球 声音 ',
 '调适 流动性 疫情 防控 中 反复 琢磨 严控 疫情 最小 代价 病毒 传播速度 掩盖 治病 率低 传播 带来 恐慌 外省市 病毒 不太 病毒 治愈 治愈 国家 定位 经济 民生 中 国家 承担 责任 定位 圈子里 篇文章 深有感触 肩负 越 做事 越 束缚 走起路来 就会慢 ',
 '信息 这变 接到 上海市 防控 办 徐汇区 防控 办 昨天 六院 复查 除了 核酸 大筛 大门 二门 不迈 六院 情况 ',
 '大亚湾 防疫 疫情 改变 事情 疫情 苏州 惠州 深圳 珠海 澳门 深圳 香港 两地 通勤 生活 疫情 一衣带水 两

In [27]:
u_emoji = []
for line3 in out:
    line_u = str(line3)
    line_un = emoji.replace_emoji(line_u,' ') # 这里的返回值是字符串
    line_u = re.sub('[a-zA-Z]',' ',line_un) #去掉英文字母
    line_f = re.sub(r'[0-9]+', ' ', line_u) #去掉数字
    #line_ff = re.sub('([^\u4e00-\u9fa5])', ' ', line_f) #去掉除中文外全部字符
    u_emoji.append(line_f)

In [28]:
u_emoji

['',
 '十点 停电 国家电网 电力 电话 女房东 懂 电路 男 房东 不上 小区 封 好惨 热 天 停电 ',
 '电影 人生大事 想 看个 电影 距离 劝退 周边 影院 歇业 ',
 '  天 居家 隔离 吃饭 ',
 '昨天 通知 做 核酸 做 治疗 八点半 车来 下楼 两个 大白 穿 隔离 服 说 阳性 病人 这栋 楼有 阳性 接触 自觉 躲避 大白 一闹 司机 阳性 吓跑 随后 两 小时 一辆车 来接 说 走 走 妈妈 走 远 护士 非常 惊讶 帮 安排 检测 召唤 一辆 转运 车 十二点 回到 家 大白 理解 阴性 非 阻止 就医 实在 不可理喻 封 控楼 阴性 住户 配合 被关 歧视 ',
 '平平安安 健健康康 ',
 '奉行 吃 想 吃 饭 想见 喜欢 风景 做 喜欢 事 喜欢 地方 不留 遗憾 遗憾 想 做 ',
 '想到 留学生 网红 伤害 ',
 '补 徐汇区 漏 闵行区 补 闵行区 漏 嘉定区 补 嘉定区 漏 浦东新区 轮流 补 一圈 ',
 '英国 做 三轮 化疗 大夫 委婉 拒绝 回国 飞机 绿色通道 进 医院 专家组 远程 开会 病人 一整套 专家组 进驻 隔离区 实地 会诊 专家组 手上 国内 癌症 患者 出不来 舆论 施压 相当于 哭 孩子 有奶 吃 首先 舆论 闹大 网民 冲锋陷阵 政府 施压 想要 优待 没钱 特权 普通人 求助无门 雨 中 跪 祈求 别提 见到 专家 只能 闭环 治疗 网络版 医闹 祝 早日康复 陆家嘴 ',
 '蹲个 点 宝山 退出 决赛圈 救命 ',
 '吊龙 茶百道 生椰绝 绝子 周末 愉快 忘 开心 事情 ',
 '闲着 无聊 看看书 小视频 听听 击球 声音 ',
 '调适 流动性 疫情 防控 中 反复 琢磨 严控 疫情 最小 代价 病毒 传播速度 掩盖 治病 率低 传播 带来 恐慌 外省市 病毒 不太 病毒 治愈 治愈 国家 定位 经济 民生 中 国家 承担 责任 定位 圈子里 篇文章 深有感触 肩负 越 做事 越 束缚 走起路来 就会慢 ',
 '信息 这变 接到 上海市 防控 办 徐汇区 防控 办 昨天 六院 复查 除了 核酸 大筛 大门 二门 不迈 六院 情况 ',
 '大亚湾 防疫 疫情 改变 事情 疫情 苏州 惠州 深圳 珠海 澳门 深圳 香港 两地 通勤 生活 疫情 一衣带水 两地

In [29]:
'''
out = []
for line in cleaned_content:
    line_seg = seg_sentence(line) # 这里的返回值是字符串
    out.append(line_seg)
'''

'\nout = []\nfor line in cleaned_content:\n    line_seg = seg_sentence(line) # 这里的返回值是字符串\n    out.append(line_seg)\n'

In [30]:
'''
u_emoji = []
for line in out:
    line_u = str(line)
    line_un = emoji.replace_emoji(line_u,' ') # 这里的返回值是字符串
    line_u = re.sub('[a-zA-Z]',' ',line_un) #去掉英文字母
    line_f = re.sub(r'[0-9]+', ' ', line_u) #去掉数字
    #line_ff = re.sub('([^\u4e00-\u9fa5])', ' ', line_f) #去掉除中文外全部字符
    u_emoji.append(line_f)
'''

"\nu_emoji = []\nfor line in out:\n    line_u = str(line)\n    line_un = emoji.replace_emoji(line_u,' ') # 这里的返回值是字符串\n    line_u = re.sub('[a-zA-Z]',' ',line_un) #去掉英文字母\n    line_f = re.sub(r'[0-9]+', ' ', line_u) #去掉数字\n    #line_ff = re.sub('([^一-龥])', ' ', line_f) #去掉除中文外全部字符\n    u_emoji.append(line_f)\n"

In [31]:
#u_emoji

In [32]:
name = ['cut_content']
test2=pd.DataFrame(columns=name,data=u_emoji)

In [33]:
df=pd.concat([df1,test2],axis=1)
df.head()

,time,user_name,content,school,gender,birthday,created_at,content_clean,cut_content
0,2022/3/23 1:56,青春有你2-tora酱,！！！//@白日夣患者:<U+0001F64F>#上海疫情# <U+0001F64F>#闵行...,NaN,"\f\""""","\1923-07-10 巨蟹座\""""","\2014-03-14 18:53:37\""""",",",
1,2022/7/11 0:43,长肉有点难nn,！！晚上十点多停电了，国家电网和上海电力的电话一直打不进去，女房东不懂电路，男房东联系不上，...,NaN,"\f\""""","\1995-06-15 双子座\""""","\2017-01-11 22:35:41\""""",",晚上十点多停电了,国家电网和上海电力的电话一直打不进去,女房东不懂电路,男房东联系不上,小...",十点 停电 国家电网 电力 电话 女房东 懂 电路 男 房东 不上 小区 封 好惨 热 天 停电
2,2022/7/14 11:35,麦和的人字拖,# 电影人生大事##上海疫情# 我就想看个电影，距离劝退，周边的影院都还歇业呢 <U+200B>,NaN,"\f\""""","\1900-06-12 双子座\""""","\2014-03-13 21:14:40\""""",",电影人生大事,我就想看个电影,距离劝退,周边的影院都还歇业呢",电影 人生大事 想 看个 电影 距离 劝退 周边 影院 歇业
3,2022/3/26 11:40,Ponytailsweet,#2岁# 2岁14天#上海疫情# 居家隔离 每天都要好好吃饭 2上海 <U+200B>,NaN,"\f\""""","\1992-08-14 狮子座\""""","\2010-03-01 20:07:44\""""","2岁14天,居家隔离,每天都要好好吃饭,",天 居家 隔离 吃饭
4,2022/3/24 10:34,白桃甜酒么么君,#moの摸鱼日常##隔离日记##上海疫情# 昨天被通知要去做核酸准备今天做治疗。到了晚上八点...,NaN,"\f\""""","\1900-10-23 天秤座\""""","\2010-04-13 23:29:28\""""","昨天被通知要去做核酸准备今天做治疗,到了晚上八点半车来了,下楼被两个大白要求穿隔离服,说我们...",昨天 通知 做 核酸 做 治疗 八点半 车来 下楼 两个 大白 穿 隔离 服 说 阳性 病人...


In [ ]:
df.to_csv('D:/大数据研究/转年龄.csv',index=False)

##词频统计——jieba

##词频统计——jieba

In [ ]:
#统计词频

In [ ]:
df['gender'].value_counts()

In [ ]:
import jieba.analyse

In [ ]:
tword=[]
for i in range(len(df['cut_content'])):
    tar=df.loc[i,'cut_content']
    print('tar',tar)
    tfw=jieba.analyse.extract_tags(sentence=tar,topK=5)
    tword.append(tfw)
    print(tfw)
 

In [ ]:
tword

In [ ]:
count ={}
for sen in tword:
    for w in sen:
        if len(w) < 1:  # 排除单个字的干扰，使得输出结果为词语
            continue
        else:
            count[w] = count.get(w, 0) + 1  # 如果字典里键为 word 的值存在，则返回键的值并加一，如果不存在键word，则返回0再加上1

exclude = []  # 建立无关词语列表
for key in list(count.keys()):  # 遍历字典的所有键，即所有word
    if key in exclude:
        del count[key]  # 删除字典中键为无关词语的键值对
listt = list(count.items())         # 将字典的所有键值对转化为列表
listt.sort(key=lambda x: x[1], reverse=True)     # 对列表按照词频从大到小的顺序排序

for i in range(100): 
    word, number = listt[i]
    print("关键字：{:-<10}频次：{:+>8}".format(word, number))

#情感分析——情感词典


In [ ]:
#参考https://github.com/hiDaDeng/cntext/基于大连理工词典的情感分析

In [ ]:
#pip install mittens

In [ ]:
#pip install pyecharts

In [ ]:
#pip install gensim --user

In [ ]:
#pip install cnsenti

In [ ]:
from cnsenti import Sentiment
senti=Sentiment()

In [ ]:
'''for line in df['content_clean']:
    resultclean=senti.sentiment_calculate(line)
    print(resultclean)'''

In [ ]:
sentidf = pd.DataFrame()
from cnsenti import Emotion
emo=Emotion()
for line in df['content_clean']:
    emoclean=emo.emotion_count(line)
    print(emoclean)
    sentidf=sentidf.append(emoclean,ignore_index=True)

In [ ]:
'''from cnsenti import Sentiment
senti=Sentiment()
for i in range(len(df['cut_content'])):
    line=df.loc[i,'cut_content']
    res=senti.sentiment_calculate(line)
    print(res)'''

In [ ]:
'''sentidf = pd.DataFrame()
from cnsenti import Emotion
emo=Emotion()
for i in range(len(df['cut_content'])):
    line=df.loc[i,'cut_content']
    resu=emo.emotion_count(line)
    print(resu)
    sentidf=sentidf.append(resu,ignore_index=True)'''

In [ ]:
'''sentidf = pd.DataFrame()
for i in range(len(df['content'])):
    line=df.loc[i,'content']
    resu=emo.emotion_count(line)
    print(resu)
    sentidf=sentidf.append(resu,ignore_index=True)'''

In [ ]:
#indexs = ['wlength','senlength', '好', '乐', '哀', '怒','惧','恶','惊']
sentidf.head()

# 情感分析（收获库）

In [ ]:
'''sent_dict = ht.build_sent_dict(textclean,min_times=1,scale="+-1") #无种子词
for line in df['content_clean']:
    sentiout = ht.analyse_sent(line)
    print(sentiout)'''

#情感分析（百度接口） 

In [ ]:
#pip install baidu-aip

In [ ]:
from aip import AipNlp

""" 你的 APPID AK SK """
APP_ID = '32628923'
API_KEY = 'QjgaypBSO867Wk1xNMMwQC1j'
SECRET_KEY = '9lsX2hvcImeRvr0a6buF5mpINAzv5wYl'

client = AipNlp(APP_ID, API_KEY, SECRET_KEY)
text = "凌晨隔壁邻居发微信说他们楼被封了，有密接等做核酸，第一次感觉疫情离自己这么近。半夜囤了好多食材，希望疫情早日散去 2上海·康桥"

""" 调用情感倾向分析 """
dict = client.sentimentClassify(text)
print(dict)


In [ ]:
'''sentimentBD = pd.DataFrame()
import time
time.sleep(2)
for line in u_clean[:5]:
    line.encode(encoding='gbk')
    sentiBD=client.sentimentClassify(line) 
    print(sentiBD)
    df222=sentimentBD.append(sentiBD,ignore_index=True)
    df111=pd.DataFrame(sentiBD,index=[0])
'''

In [ ]:
import pandas as pd
import time
import datetime

from aip import AipNlp

import codecs

#startdate = datetime.date(2017, 8, 1).strftime('%Y-%m-%d')

#enddate = datetime.date(2018, 3, 27).strftime('%Y-%m-%d')

APP_ID = '32628923'
API_KEY = 'QjgaypBSO867Wk1xNMMwQC1j'
SECRET_KEY = '9lsX2hvcImeRvr0a6buF5mpINAzv5wYl'

client = AipNlp(APP_ID, API_KEY, SECRET_KEY)
time.sleep(2)
def get_sentiments(text):

    try:

        sitems=client.sentimentClassify(text)['items'][0]#情感分析
        textt=client.sentimentClassify(text)['text']
        negative=sitems['negative_prob']

        positive=sitems['positive_prob']#积极概率

        confidence=sitems['confidence']#置信度

        sentiment=sitems['sentiment']#0表示消极，1表示中性，2表示积极

        #tagitems = client.commentTag(text, {'type': 9}) # 评论观点

        #propertys=tagitems['prop']#属性

        #adj=tagitems['adj']#描述词

        output='{}\t{}\t{}\t{}\t{}\n'.format(textt,negative,positive,confidence,sentiment)

        f=codecs.open('sentiBD.xls','a+','utf-8')

        f.write(output)

        f.close()

        print('Done')

    except Exception as e:

        print(e)


for line in df['content_clean']:
    get_sentiments(line)

##情感分析（snownlp库）

##情感分析（snownlp库）

In [ ]:
#pip install baidu-aip

In [ ]:
import snownlp

In [ ]:
from snownlp import SnowNLP
sum=0
count=0
        
for i in range(len(df['cut_content'])):
    line=df.loc[i,'cut_content']
    words=''
    for seg in line:
        words=words+seg
    if len(words)!=0:
        print(words)   
        d=SnowNLP(words)
        print('{}'.format(d.sentiments))
        df.loc[i,'sentiment1_score']=float(d.sentiments)    
        sum+=d.sentiments
        count+=1
score=sum/count
print('finalscore={}'.format(score))   

In [ ]:
df['sentiment1_score']

In [ ]:
# 情感判定 
tag_list = [] 
neg_count=0
pos_count=0
mid_count=0
for score in df['sentiment1_score']:

    if score < 0.5:
        tag = '消极'
        neg_count += 1
    elif score > 0.5:
        tag = '积极'
        pos_count += 1
    else:
        tag = '中性'
        mid_count += 1

    tag_list.append(tag)  # 判定结果

df['tag'] = tag_list

In [ ]:
grp = df['tag'].value_counts()
print('正负面评论统计：')
print(grp)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     

In [ ]:
#情感分析可视化

In [ ]:
df_p=pd.concat([df,sentidf],axis=1)
df_p.head()

In [ ]:
df_p['day'] = df_p['time'].str[:9]
df_p.head()

In [ ]:
#data = df_p.groupby('day')['sentiment1_score'].mean()
#data

# 描述性统计

In [ ]:
import pandas as pd 
import re
import datetime

list1 = ['birthday']	#将列名填入列表以作循环
for i in list1:
    df_p[i] = df_p[i].astype(str)   #转换数据类型
#    def clear_characters(text):
#        return re.sub('[\u4e00-\u9fa5]', "", text)		#设置规则1，替换成空格
#    def clear_characters1(text):
#        return re.sub(r'\\',"", text)	#设置规则2
    def clear_characters2(text):
        return re.sub(r'\D',"", text)
    def clear_characters1(text):
        return re.sub('19840300|19950000',"", text)
	#设置规则3
#    #循环列名替换特殊符号
#    tt[i] = tt[i].apply(clear_characters1)
    df_p[i] = df_p[i].apply(clear_characters2)
    df_p[i] = df_p[i].apply(clear_characters1) 

print(df_p['birthday'])


In [ ]:
import time
df_p['birthday'] =pd.to_datetime(df_p['birthday'])
print(df_p['birthday'])

In [ ]:
import datetime as dt
now_year =dt.datetime.today().year #当前的年份
df_p['age']=now_year-df_p.birthday.dt.year
df_p

In [ ]:
vc=df_p['age'].value_counts()  # 频数统计
print(vc)

In [ ]:
vcc=df_p['day'].value_counts().rename_axis('day').reset_index(name='counts')
print(vcc)

In [ ]:
type(vcc)

In [ ]:
d_count=d.rename(columns={'day':'day','好':'good','乐':'joy','哀':'sadness','怒':'anger','惧':'fear','恶':'disgust','惊':'surprise'})

In [ ]:
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
%matplotlib inline
plt.rcParams['font.sans-serif'] = ['SimHei']  # 设置加载的字体名
plt.rcParams['axes.unicode_minus'] = False    # 解决保存图像是负号'-'显示为方块的问题
sns.set_style('white')   #设置图形背景样式为whiteimport seaborn as sns

In [ ]:
#sns.displot(df_p['sentiment1_score'],bins=30,kde=True)

In [ ]:
'''fig,ax1=plt.subplots(figsize=(40,20))
#绘制基础图层
color = 'tab:green'
ax1.set_title('情绪值变化', fontsize=16)
ax1.set_xlabel('day', fontsize=16)
ax1 = sns.lineplot(x='day', y='好', data = df_p, palette='summer')
ax1.tick_params(axis='y')
#twinx共享x轴(类似的语法，如共享y轴twiny)
ax2 = ax1.twinx()
color = 'tab:red'
ax2 = sns.lineplot(x='day', y='乐', data = df_p, sort=False, color=color)
ax2.tick_params(axis='y', color=color)

ax3 = ax1.twinx()
color = 'tab:blue'
ax3 = sns.lineplot(x='day', y='哀', data = df_p, sort=False, color=color)
ax3.tick_params(axis='y', color=color)

ax4 = ax1.twinx()
color = 'tab:pink'
ax4 = sns.lineplot(x='day', y='怒', data = df_p, sort=False, color=color)
ax4.tick_params(axis='y', color=color)

ax5 = ax1.twinx()
color = 'tab:purple'
ax5 = sns.lineplot(x='day', y='惧', data = df_p, sort=False, color=color)
ax5.tick_params(axis='y', color=color)

ax6 = ax1.twinx()
ax6 = sns.lineplot(x='day', y='恶', data = df_p, sort=False)
ax6.tick_params(axis='y', color=color)

ax7 = ax1.twinx()
ax7 = sns.lineplot(x='day', y='惊', data = df_p, sort=False)
ax7.tick_params(axis='y', color=color)

#显示绘制结果
plt.show()
'''

In [ ]:
#d_hao = df_p.groupby('day')['好', '乐', '哀', '怒','惧','恶','惊'].sum().reset_index(name=['good_counts','joy_counts','sadness_counts','anger_counts','fear_counts','disgust_counts','surprise_counts'])
d = df_p.groupby('day')['好', '乐', '哀', '怒','惧','恶','惊'].sum()
d

In [ ]:
d_count=d.rename(columns={'day':'day','好':'good','乐':'joy','哀':'sadness','怒':'anger','惧':'fear','恶':'disgust','惊':'surprise'})
d_count

In [ ]:
plt.rcParams['font.sans-serif'] = ['simHei'] #用来正常显示中文标签
plt.rcParams['axes.unicode_minus'] = False   #用来正常显示负号

d_count.plot()
plt.savefig(u'test.png', dpi=500)
plt.show()

In [ ]:
# 设置显示中文
plt.rcParams['font.sans-serif'] = ['KaiTi']
plt.rcParams['axes.unicode_minus'] = False
plt.style.use('ggplot')
d_count.plot(kind='bar')

In [ ]:
d_count.plot(kind='bar', stacked=True)
plt.show()

In [ ]:
d_count.plot(subplots=True)
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(12, 8), dpi=80)
sns.lineplot(x='day', y='good', data = d_count)

# 装饰图片
plt.rcParams['font.sans-serif'] = ['Simhei']  # 设置字体为黑体
plt.xlabel('day', fontsize=16)  # x轴名称
plt.ylabel('good', fontsize=16)  # y轴名称
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
plt.title('好随时间变化图', fontsize=20)
plt.show()

In [ ]:
d1 = df_p.groupby('day')['好', '乐', '哀', '怒','惧','恶','惊'].mean()
d1

In [ ]:
plt.rcParams['font.sans-serif'] = ['simHei'] #用来正常显示中文标签
plt.rcParams['axes.unicode_minus'] = False   #用来正常显示负号

d1.plot()
plt.savefig(u'test.png', dpi=500)
plt.show()

In [ ]:
d1.plot(kind='bar', stacked=True)
plt.show()

In [ ]:
d_age = df_p.groupby('age')['好', '乐', '哀', '怒','惧','恶','惊'].sum()
d_age

In [ ]:
dage=d_age.rename(columns={'day':'day','好':'good','乐':'joy','哀':'sadness','怒':'anger','惧':'fear','恶':'disgust','惊':'surprise'})
dage

In [ ]:
plt.rcParams['font.sans-serif'] = ['simHei'] #用来正常显示中文标签
plt.rcParams['axes.unicode_minus'] = False   #用来正常显示负号

dage.plot()
plt.show()

# 相关分析

In [ ]:
dfill = pd.read_excel(r'D:/0609-3/大数据研究/新增病例.xlsx')

In [ ]:
dfill.head()

In [ ]:
mergee = pd.merge(d_count, dfill, on='day',how='outer') 
merge = pd.merge(mergee, vcc, on='day',how='outer') 
# 其中 join_axes 参数为指定根据哪个索引进行数据对齐
# join_axes=[source_df.index] 则根据

In [ ]:
merge

In [ ]:
corr = merge[['good','joy','sadness','anger','fear','disgust','surprise','SHconfirmedcases','SHasymptomatic','SHtotal','counts']].corr()
print(corr)

In [ ]:
sns.heatmap(corr,xticklabels=corr.columns,yticklabels=corr.columns)
plt.show()

# 相关分析

In [ ]:
dfill = pd.read_excel(r'D:/0609-3/大数据研究/新增病例.xlsx')

In [ ]:
dfill.head()

In [ ]:
mergee = pd.merge(d_count, dfill, on='day',how='outer') 
merge = pd.merge(mergee, vcc, on='day',how='outer') 
# 其中 join_axes 参数为指定根据哪个索引进行数据对齐
# join_axes=[source_df.index] 则根据

In [ ]:
merge

In [ ]:
corr = merge[['good','joy','sadness','anger','fear','disgust','surprise','SHconfirmedcases','SHasymptomatic','SHtotal','counts']].corr()
print(corr)

In [ ]:
sns.heatmap(corr,xticklabels=corr.columns,yticklabels=corr.columns)
plt.show()

##主题分析

In [4]:
tt = pd.read_csv(r'D:/0609-3/大数据研究/dataed.csv',encoding='utf-8')

In [9]:
tt.head()

,time,user_name,content,school,gender,birthday,created_at,content_clean,cut_content
0,2022/3/23 1:56,青春有你2-tora酱,！！！//@白日夣患者:<U+0001F64F>#上海疫情# <U+0001F64F>#闵行...,NaN,"\f\""""","\1923-07-10 巨蟹座\""""","\2014-03-14 18:53:37\""""",",",NaN
1,2022/7/11 0:43,长肉有点难nn,！！晚上十点多停电了，国家电网和上海电力的电话一直打不进去，女房东不懂电路，男房东联系不上，...,NaN,"\f\""""","\1995-06-15 双子座\""""","\2017-01-11 22:35:41\""""",",晚上十点多停电了,国家电网和上海电力的电话一直打不进去,女房东不懂电路,男房东联系不上,小...",十点 停电 国家电网 电力 电话 女房东 懂 电路 男 房东 不上 小区 封 好惨 热 天 停电
2,2022/7/14 11:35,麦和的人字拖,# 电影人生大事##上海疫情# 我就想看个电影，距离劝退，周边的影院都还歇业呢 <U+200B>,NaN,"\f\""""","\1900-06-12 双子座\""""","\2014-03-13 21:14:40\""""",",电影人生大事,我就想看个电影,距离劝退,周边的影院都还歇业呢",电影 人生大事 想 看个 电影 距离 劝退 周边 影院 歇业
3,2022/3/26 11:40,Ponytailsweet,#2岁# 2岁14天#上海疫情# 居家隔离 每天都要好好吃饭 2上海 <U+200B>,NaN,"\f\""""","\1992-08-14 狮子座\""""","\2010-03-01 20:07:44\""""","2岁14天,居家隔离,每天都要好好吃饭,",天 居家 隔离 吃饭
4,2022/3/24 10:34,白桃甜酒么么君,#moの摸鱼日常##隔离日记##上海疫情# 昨天被通知要去做核酸准备今天做治疗。到了晚上八点...,NaN,"\f\""""","\1900-10-23 天秤座\""""","\2010-04-13 23:29:28\""""","昨天被通知要去做核酸准备今天做治疗,到了晚上八点半车来了,下楼被两个大白要求穿隔离服,说我们...",昨天 通知 做 核酸 做 治疗 八点半 车来 下楼 两个 大白 穿 隔离 服 说 阳性 病人...


In [5]:
import re
import numpy as np
import pandas as pd
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
 
# spacy for lemmatization
import spacy
 
# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

from gensim import corpora
import matplotlib
from gensim.models.coherencemodel import CoherenceModel
from gensim.models.ldamodel import LdaModel

In [10]:
tt.isnull().any(axis=0)

time             False
user_name        False
content          False
school            True
gender            True
birthday         False
created_at        True
content_clean     True
cut_content       True
dtype: bool

In [11]:
ttda = tt.dropna(axis=0,subset=['cut_content'])

In [12]:
ttda

,time,user_name,content,school,gender,birthday,created_at,content_clean,cut_content
1,2022/7/11 0:43,长肉有点难nn,！！晚上十点多停电了，国家电网和上海电力的电话一直打不进去，女房东不懂电路，男房东联系不上，...,NaN,"\f\""""","\1995-06-15 双子座\""""","\2017-01-11 22:35:41\""""",",晚上十点多停电了,国家电网和上海电力的电话一直打不进去,女房东不懂电路,男房东联系不上,小...",十点 停电 国家电网 电力 电话 女房东 懂 电路 男 房东 不上 小区 封 好惨 热 天 停电
2,2022/7/14 11:35,麦和的人字拖,# 电影人生大事##上海疫情# 我就想看个电影，距离劝退，周边的影院都还歇业呢 <U+200B>,NaN,"\f\""""","\1900-06-12 双子座\""""","\2014-03-13 21:14:40\""""",",电影人生大事,我就想看个电影,距离劝退,周边的影院都还歇业呢",电影 人生大事 想 看个 电影 距离 劝退 周边 影院 歇业
3,2022/3/26 11:40,Ponytailsweet,#2岁# 2岁14天#上海疫情# 居家隔离 每天都要好好吃饭 2上海 <U+200B>,NaN,"\f\""""","\1992-08-14 狮子座\""""","\2010-03-01 20:07:44\""""","2岁14天,居家隔离,每天都要好好吃饭,",天 居家 隔离 吃饭
4,2022/3/24 10:34,白桃甜酒么么君,#moの摸鱼日常##隔离日记##上海疫情# 昨天被通知要去做核酸准备今天做治疗。到了晚上八点...,NaN,"\f\""""","\1900-10-23 天秤座\""""","\2010-04-13 23:29:28\""""","昨天被通知要去做核酸准备今天做治疗,到了晚上八点半车来了,下楼被两个大白要求穿隔离服,说我们...",昨天 通知 做 核酸 做 治疗 八点半 车来 下楼 两个 大白 穿 隔离 服 说 阳性 病人...
5,2022/3/21 22:25,九二一哈,#mu5735##上海疫情# <U+0001F64F>平平安安，健健康康~ <U+200B>,NaN,"\f\""""","\1997-05-23 双子座\""""","\2015-10-15 19:24:02\""""","平平安安,健健康康,",平平安安 健健康康
...,...,...,...,...,...,...,...,...,...
25286,2022/3/4 18:58,谷市大白话,#上海疫情# 病例1：男，48岁，司机，居住于闵行区普乐路333弄，为3月3日报告的松江区永...,NaN,"\m\""""","\狮子座\""""","\2011-02-04 11:00:54\""""","病例1,男,48岁,司机,居住于闵行区普乐路333弄,为3月3日报告的松江区永辉超市沪亭北路...",病例 男 司机 居住 闵行区 普乐 路 弄 报告 松江区 永辉 超市 沪 亭 北路 ...
25287,2022/3/4 9:18,FEELING平安喜乐,#上海疫情# 上海14例无症状：其中普陀区9例、松江区3例、宝山区2例。 <U+200B>,NaN,"\f\""""","\金牛座\""""","\2011-04-24 18:25:15\""""","上海14例无症状,其中普陀区9例,松江区3例,宝山区2例,",无症状 普陀区 松江区 宝山区
25288,2022/3/4 8:43,爱上海201188,【上海市第十人民医院延长中路院区暂停医疗服务】 上海十院昨晚发布公告，根据上海市疫情防控要求...,NaN,"\f\""""","\天蝎座\""""","\2011-12-16 21:55:27\""""",",上海市第十人民医院延长中路院区暂停医疗服务,上海十院昨晚发布公告,根据上海市疫情防控要求,...",上海市 第十 医院 延长 中路 院区 暂停 医疗 服务 十院 昨晚 发布公告 上海市 疫情 ...
25289,2022/3/3 23:58,海浪泡沫Aph,海浪泡沫Aph#happywithAph# 这几天本来因为工作逐渐烦躁，突然疫情起来了，连...,NaN,"\m\""""","\天秤座\""""","\2010-03-14 01:54:18\""""",",海浪泡沫,这几天本来因为工作逐渐烦躁,突然疫情起来了,连自己小区里都有了一例密接,封了一栋...",海浪 泡沫 几天 工作 烦躁 疫情 小区 里 一例 密接 封 一栋 楼 明早 出门 上班 万...


In [13]:
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
 
# spacy for lemmatization
import spacy
 
# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

from gensim import corpora
import matplotlib
from gensim.models.coherencemodel import CoherenceModel
from gensim.models.ldamodel import LdaModel

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation

In [14]:
from sklearn.feature_extraction.text import CountVectorizer
vect = CountVectorizer()
term_matrix = pd.DataFrame(vect.fit_transform(ttda['cut_content']).toarray(), columns=vect.get_feature_names())
term_matrix.head()

C:\Program Files\Python310\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,一丁,一丁点,一万,一万个,一万多,一万多块,一万年,一万次,一万步,一万遍,...,龙盛佳苑,龙腾,龙臣,龙茗路,龙虾,龙阳路,龙飞,龚嘉轩,龚正,龚路
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [29]:
max_df = 0.8 # 在超过这一比例的文档中出现的关键词（过于平凡），去除掉。
min_df = 3 # 在低于这一数量的文档中出现的关键词（过于独特），去除掉。

vect = CountVectorizer(max_df = max_df, 
                       min_df = min_df, 
                       token_pattern=u'(?u)\\b[^\\d\\W]\\w+\\b', 
                       stop_words=frozenset(stopwords))

term_matrix = pd.DataFrame(vect.fit_transform(ttda.cut_content).toarray(), columns=vect.get_feature_names())
term_matrix.head()

C:\Program Files\Python310\lib\site-packages\sklearn\feature_extraction\text.py:404: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['lex'] not in stop_words.
  warnings.warn(
C:\Program Files\Python310\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,一丁,一万,一万个,一万多,一万步,一上午,一下下,一下子,一不小心,一丝,...,龙之梦,龙华,龙吴,龙城,龙州,龙柏,龙潭,龙腾,龙虾,龚正
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [31]:
# 构造 TF-IDF
tf_idf_vectorizer = TfidfVectorizer()
tf_idf = tf_idf_vectorizer.fit_transform(ttda['cut_content'])

In [32]:
# 特征词列表
feature_names = tf_idf_vectorizer.get_feature_names()
# 特征词 TF-IDF 矩阵
#matrix = tf_idf.toarray()
matrix =term_matrix
feature_names_df = pd.DataFrame(matrix,columns=feature_names)
print(feature_names_df)
feature_names_df

C:\Program Files\Python310\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


       一丁  一丁点  一万  一万个  一万多  一万多块  一万年  一万次  一万步  一万遍  ...  龙盛佳苑  龙腾  龙臣  \
0       0  NaN   0    0    0   NaN  NaN  NaN    0  NaN  ...   NaN   0 NaN   
1       0  NaN   0    0    0   NaN  NaN  NaN    0  NaN  ...   NaN   0 NaN   
2       0  NaN   0    0    0   NaN  NaN  NaN    0  NaN  ...   NaN   0 NaN   
3       0  NaN   0    0    0   NaN  NaN  NaN    0  NaN  ...   NaN   0 NaN   
4       0  NaN   0    0    0   NaN  NaN  NaN    0  NaN  ...   NaN   0 NaN   
...    ..  ...  ..  ...  ...   ...  ...  ...  ...  ...  ...   ...  ..  ..   
23849   0  NaN   0    0    0   NaN  NaN  NaN    0  NaN  ...   NaN   0 NaN   
23850   0  NaN   0    0    0   NaN  NaN  NaN    0  NaN  ...   NaN   0 NaN   
23851   0  NaN   0    0    0   NaN  NaN  NaN    0  NaN  ...   NaN   0 NaN   
23852   0  NaN   0    0    0   NaN  NaN  NaN    0  NaN  ...   NaN   0 NaN   
23853   0  NaN   0    0    0   NaN  NaN  NaN    0  NaN  ...   NaN   0 NaN   

       龙茗路  龙虾  龙阳路  龙飞  龚嘉轩  龚正  龚路  
0      NaN   0  NaN NaN  NaN   0 NaN

,一丁,一丁点,一万,一万个,一万多,一万多块,一万年,一万次,一万步,一万遍,...,龙盛佳苑,龙腾,龙臣,龙茗路,龙虾,龙阳路,龙飞,龚嘉轩,龚正,龚路
0,0,NaN,0,0,0,NaN,NaN,NaN,0,NaN,...,NaN,0,NaN,NaN,0,NaN,NaN,NaN,0,NaN
1,0,NaN,0,0,0,NaN,NaN,NaN,0,NaN,...,NaN,0,NaN,NaN,0,NaN,NaN,NaN,0,NaN
2,0,NaN,0,0,0,NaN,NaN,NaN,0,NaN,...,NaN,0,NaN,NaN,0,NaN,NaN,NaN,0,NaN
3,0,NaN,0,0,0,NaN,NaN,NaN,0,NaN,...,NaN,0,NaN,NaN,0,NaN,NaN,NaN,0,NaN
4,0,NaN,0,0,0,NaN,NaN,NaN,0,NaN,...,NaN,0,NaN,NaN,0,NaN,NaN,NaN,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23849,0,NaN,0,0,0,NaN,NaN,NaN,0,NaN,...,NaN,0,NaN,NaN,0,NaN,NaN,NaN,0,NaN
23850,0,NaN,0,0,0,NaN,NaN,NaN,0,NaN,...,NaN,0,NaN,NaN,0,NaN,NaN,NaN,0,NaN
23851,0,NaN,0,0,0,NaN,NaN,NaN,0,NaN,...,NaN,0,NaN,NaN,0,NaN,NaN,NaN,0,NaN
23852,0,NaN,0,0,0,NaN,NaN,NaN,0,NaN,...,NaN,0,NaN,NaN,0,NaN,NaN,NaN,0,NaN


In [ ]:
# 指定 lda 主题数
n_topics = 5
# 要输出的每个主题的前 n_top_words 个主题词数
n_top_words = 20

lda = LatentDirichletAllocation(
    n_components=n_topics, max_iter=100,
    learning_method='online',
    learning_offset=50.,
    random_state=0)
# n_components 主题数量 learning_method更新主题的方法（online VS batch） learning_offset一个正参数，可以减轻早期迭代的负担
#max_iter迭代的最大次数
# 核心，给 LDA 喂生成的 TF-IDF 矩阵
lda.fit(tf_idf)

In [ ]:
def top_words_data_frame(model: LatentDirichletAllocation,
                         tf_idf_vectorizer: TfidfVectorizer,
                         n_top_words: int) -> pd.DataFrame:
    '''
    求出每个主题的前 n_top_words 个词

    Parameters
    ----------
    model : sklearn 的 LatentDirichletAllocation 
    tf_idf_vectorizer : sklearn 的 TfidfVectorizer
    n_top_words :前 n_top_words 个主题词

    Return
    ------
    DataFrame: 包含主题词分布情况
    '''
    rows = []
    feature_names = tf_idf_vectorizer.get_feature_names()
    for topic in model.components_:
        top_words = [feature_names[i]
                     for i in topic.argsort()[:-n_top_words - 1:-1]]
        rows.append(top_words)
    columns = [f'topic {i+1}' for i in range(n_top_words)]
    df = pd.DataFrame(rows, columns=columns)

    return df


def predict_to_data_frame(model: LatentDirichletAllocation, X: np.ndarray) -> pd.DataFrame:
    '''
    求出文档主题概率分布情况

    Parameters
    ----------
    model : sklearn 的 LatentDirichletAllocation 
    X : 词向量矩阵

    Return
    ------
    DataFrame: 包含主题词分布情况
    '''
    # 求出给定文档的主题概率分布矩阵
    matrix = model.transform(X)
    columns = [f'P(topic {i+1})' for i in range(len(model.components_))]
    df = pd.DataFrame(matrix, columns=columns)
    return df

In [ ]:
# 计算 n_top_words 个主题词
top_words_df = top_words_data_frame(lda, tf_idf_vectorizer, n_top_words)
top_words_df
# 保存 n_top_words 个主题词到 csv 文件中
#top_words_df.to_csv(top_words_csv_path, encoding='utf-8-sig', index=None)

# 转 tf_idf 为数组，以便后面使用它来对文本主题概率分布进行计算
X = tf_idf.toarray()

# 计算完毕主题概率分布情况
predict_df = predict_to_data_frame(lda, X)
predict_df


In [ ]:
top_words_df

In [33]:
#完善停用词表
newstop = (['顾魏','特别','越来越','好像','我要','刚刚','天天','林之校','啊啊啊','王一博','翟潇闻','真的','哈哈哈','感觉',
       '不到','一点','呜呜','哈哈哈哈','好好','这是','时代','少年','余生','指教','本来','晚上','天天','不想','刘浩存','电视剧','）','（','.','。','.#','#','','：',''])

In [42]:
aaa = []
for line in ttda['cut_content']:
    line_u1 = ' '.join(str(line).split())
    line_uu1 = line_u1.split()#将一个元素按空格拆分成多个元素
    outstr1 = []
    for words in line_uu1:
        if words not in newstop:
            #if words in ['n','v','a','r','i']:
                outstr1.append(words)
    aaa.append(outstr1)

In [43]:
while [] in aaa:
    aaa.remove([])

In [44]:
aaa

[['十点',
  '停电',
  '国家电网',
  '电力',
  '电话',
  '女房东',
  '懂',
  '电路',
  '男',
  '房东',
  '不上',
  '小区',
  '封',
  '好惨',
  '热',
  '天',
  '停电'],
 ['电影', '人生大事', '想', '看个', '电影', '距离', '劝退', '周边', '影院', '歇业'],
 ['天', '居家', '隔离', '吃饭'],
 ['昨天',
  '通知',
  '做',
  '核酸',
  '做',
  '治疗',
  '八点半',
  '车来',
  '下楼',
  '两个',
  '大白',
  '穿',
  '隔离',
  '服',
  '说',
  '阳性',
  '病人',
  '这栋',
  '楼有',
  '阳性',
  '接触',
  '自觉',
  '躲避',
  '大白',
  '一闹',
  '司机',
  '阳性',
  '吓跑',
  '随后',
  '两',
  '小时',
  '一辆车',
  '来接',
  '说',
  '走',
  '走',
  '妈妈',
  '走',
  '远',
  '护士',
  '非常',
  '惊讶',
  '帮',
  '安排',
  '检测',
  '召唤',
  '一辆',
  '转运',
  '车',
  '十二点',
  '回到',
  '家',
  '大白',
  '理解',
  '阴性',
  '非',
  '阻止',
  '就医',
  '实在',
  '不可理喻',
  '封',
  '控楼',
  '阴性',
  '住户',
  '配合',
  '被关',
  '歧视'],
 ['平平安安', '健健康康'],
 ['奉行',
  '吃',
  '想',
  '吃',
  '饭',
  '想见',
  '喜欢',
  '风景',
  '做',
  '喜欢',
  '事',
  '喜欢',
  '地方',
  '不留',
  '遗憾',
  '遗憾',
  '想',
  '做'],
 ['想到', '留学生', '网红', '伤害'],
 ['补',
  '徐汇区',
  '漏',
  '闵行区',
  '补',
  '闵行区',
  '漏',
  '嘉定区',
 

In [45]:
import gensim.downloader as api
from gensim.models import TfidfModel
from gensim.corpora import Dictionary

In [46]:
dictionary = corpora.Dictionary(aaa)
corpus = [dictionary.doc2bow(tmp) for tmp in aaa]

In [47]:
tfidf = TfidfModel(corpus)
corpus_tfidf = tfidf[corpus]

In [ ]:
# 困惑度的XY坐标
x_perplexity = [] 
y_perplexity = [] 
# 一致性的XY坐标
x_coherence = [] 
y_coherence = [] 
#id2word：就是上边构造的dictionary字典
#passes：模型的训练次数
for i in range(1, 10):     
    # 循环生成主题数为i的模型， passes代表模型训练次数，数据量太大了，搞小一点。     
    lda = gensim.models.ldamodel.LdaModel(corpus_tfidf, num_topics=i, id2word=dictionary, passes=10)     
    # 计算当前模型困惑度
    cur_perplexity = lda.log_perplexity(corpus_tfidf)     
    # 构造横纵坐标轴数据
    x_perplexity.append(i)
    y_perplexity.append(cur_perplexity) 
    # 计算一致性
    cv_tmp = CoherenceModel(model=lda, texts=aaa, dictionary=dictionary, coherence='c_v') 
    x_coherence.append(i) 
    y_coherence.append(cv_tmp.get_coherence()) 

In [ ]:
# 绘制一致性折线图 
plt.plot(x_coherence, y_coherence) 
plt.xlabel('num topics') 
plt.ylabel('coherence score') 
plt.legend(('coherence_values'), loc='best') 
plt.savefig('D:\大数据研究\coherenceLine4.21.1.svg') 
plt.show() 

In [ ]:
pyLDAvis.enable_notebook()
data = pyLDAvis.gensim.prepare(lda, corpus_tfidf, dictionary)
#让可视化可以在notebook内显示
pyLDAvis.show(data)

In [ ]:
# 主题分析

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
vect = CountVectorizer()
term_matrix = pd.DataFrame(vect.fit_transform(df['cut_content']).toarray(), columns=vect.get_feature_names())
term_matrix.head()